In [1]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import tensorflow.python.keras
from tensorflow.python.keras import models, layers
from tensorflow.python.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.layers import BatchNormalization, Activation, Flatten
from tensorflow.python.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os

Using TensorFlow backend.


In [2]:
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 10
l = 40
compression = 0.55
dropout_rate = 0.2

In [3]:
# Load CIFAR10 Data

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]

# convert to one hot encoing 
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes) 

In [4]:
X_train.shape

(50000, 32, 32, 3)

In [5]:
X_test.shape

(10000, 32, 32, 3)

In [7]:
X_train_mean = np.mean(X_train, axis=(0,1,2))
X_train_std = np.std(X_train, axis=(0,1,2))
X_train = (X_train - X_train_mean) / X_train_std
X_test = (X_test - X_train_mean) / X_train_std

In [8]:
#
# Dense Block
def denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l): 
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.SeparableConv2D(int(num_filter*compression), (5,5), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
            Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

## transition Blosck
def transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.SeparableConv2D(int(num_filter*compression), (5,5), use_bias=False,padding='same')(relu)
    if dropout_rate>0:
         Conv2D_BottleNeck = layers.Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

#output layer
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(4,4))(relu)
    #flat = layers.Flatten()(AvgPooling)
    output = layers.SeparableConv2D(num_classes, (5,5), use_bias=False,padding='same', activation='softmax')(AvgPooling)
    return output

In [9]:
import keras.backend as K
K.clear_session()


In [11]:
num_filter = 126

dropout_rate = 0
l = 7

input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.SeparableConv2D(num_filter, (5,5), use_bias=False, padding='same')(input)

First_Block = denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = transition(First_Block, num_filter, dropout_rate)

Second_Block = denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = transition(Second_Block, num_filter, dropout_rate)

Third_Block = denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = transition(Third_Block, num_filter, dropout_rate)

Last_Block = denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [12]:
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
separable_conv2d (SeparableConv (None, 32, 32, 126)  453         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 126)  504         separable_conv2d[0][0]           
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 126)  0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [13]:
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.15,
    height_shift_range=0.15,
    horizontal_flip=True,
    zoom_range=0.10,
    )
datagen.fit(X_train)

In [14]:
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau, LearningRateScheduler

In [15]:
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint_1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, mode='max')


In [16]:
reduce_lr_1 = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=4,verbose = 1)

In [17]:
earlystopping_1 = EarlyStopping(monitor='val_loss', patience=40, verbose=1)

In [18]:
callbacks_list = [earlystopping_1,reduce_lr_1,checkpoint_1]

In [19]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [20]:
# reshaping to match with convoultion output layer
y_train_re = np.reshape(y_train, (50000,1,1,10))
y_test_re = np.reshape(y_test, (10000,1,1,10))

In [21]:
history = model.fit_generator(datagen.flow(X_train, y_train_re, batch_size=batch_size),
                    steps_per_epoch=X_train.shape[0] // batch_size,
                    epochs=100,
                    verbose=2, 
                    validation_data=(X_test, y_test_re),callbacks=callbacks_list)

Epoch 1/100
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 00001: saving model to weights-improvement-01-0.49.hdf5
781/781 - 115s - loss: 1.6273 - acc: 0.3934 - val_loss: 1.4025 - val_acc: 0.4926
Epoch 2/100

Epoch 00002: saving model to weights-improvement-02-0.58.hdf5
781/781 - 106s - loss: 1.1767 - acc: 0.5819 - val_loss: 1.2740 - val_acc: 0.5768
Epoch 3/100

Epoch 00003: saving model to weights-improvement-03-0.63.hdf5
781/781 - 109s - loss: 0.9644 - acc: 0.6608 - val_loss: 1.1408 - val_acc: 0.6313
Epoch 4/100

Epoch 00004: saving model to weights-improvement-04-0.70.hdf5
781/781 - 101s - loss: 0.8385 - acc: 0.7094 - val_loss: 0.9185 - val_acc: 0.6954
Epoch 5/100

Epoch 00005: saving model to weights-improvement-05-0.71.hdf5
781/781 - 101s - loss: 0.7522 - acc: 0.7385 - val_loss: 0.9352 - val_acc: 0.7057
Epoch 6/100

Epoch 00006: saving model to weights-improvement-06-0.72.hdf5
781/781 - 101s - loss: 0.6856 - acc: 0.7621 - val_l

781/781 - 101s - loss: 0.1457 - acc: 0.9489 - val_loss: 0.3012 - val_acc: 0.9083
Epoch 49/100

Epoch 00049: saving model to weights-improvement-49-0.91.hdf5
781/781 - 101s - loss: 0.1437 - acc: 0.9496 - val_loss: 0.3002 - val_acc: 0.9088
Epoch 50/100

Epoch 00050: saving model to weights-improvement-50-0.91.hdf5
781/781 - 107s - loss: 0.1438 - acc: 0.9504 - val_loss: 0.3000 - val_acc: 0.9092
Epoch 51/100

Epoch 00051: saving model to weights-improvement-51-0.91.hdf5
781/781 - 107s - loss: 0.1459 - acc: 0.9500 - val_loss: 0.3047 - val_acc: 0.9070
Epoch 52/100

Epoch 00052: ReduceLROnPlateau reducing learning rate to 1.000000082740371e-10.

Epoch 00052: saving model to weights-improvement-52-0.91.hdf5
781/781 - 102s - loss: 0.1446 - acc: 0.9495 - val_loss: 0.3031 - val_acc: 0.9076
Epoch 53/100

Epoch 00053: saving model to weights-improvement-53-0.91.hdf5
781/781 - 106s - loss: 0.1439 - acc: 0.9503 - val_loss: 0.2994 - val_acc: 0.9097
Epoch 54/100

Epoch 00054: saving model to weights-im

In [24]:
# Test the model
score = model.evaluate(X_test, y_test_re, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 5s 549us/sample - loss: 0.3026 - acc: 0.9088
Test loss: 0.3025546526670456
Test accuracy: 0.9088


In [25]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


# Summary

* Applied DenseNet architecture on CIFR10 dataset
* Tuned the architecture 
* Obtained a test accuracy of 90.88